In [ ]:
%run '/Users/itaymi/Desktop/Python/Functions.ipynb'
display(HTML('<style>.container{width:100% !important;}</style>'))

#### Dates range

In [ ]:
history = 150
from_date = (date.today() - timedelta(days = history)).strftime('%Y-%m-%d')
to_date   = date.today().strftime('%Y-%m-%d')

print(f'{from_date} to {to_date}')

#### Define team members

In [ ]:
team_members = ['Itay Milner',
                'Jenna Israelsohn',
                'Ofir Weisfeld',
                'Tal Rath',
                'Shir Zaltsman']

#### Jira API

In [ ]:
j = pd.read_csv('jira_pass.csv')

JIRA_URL, JIRA_USERNAME, JIRA_API_TOKEN = j.at[0,'jira'], j.at[1,'jira'], j.at[2,'jira']

session = requests.Session()
session.auth = (JIRA_USERNAME, JIRA_API_TOKEN)

start_at, max_results = 0, 100
issues = []

while True:
    JQL_QUERY = f'project = DATA AND created >= {from_date} AND created <= {to_date} ORDER BY created DESC'
    
    params = {'jql':JQL_QUERY, 'startAt':start_at, 'maxResults':max_results}
    
    response = session.get(f'{JIRA_URL}search', params = params)
    
    if response.status_code == 200:
        batch_issues = response.json()['issues']
        issues.extend(batch_issues) 

        if len(batch_issues) < max_results:
            break

        start_at += max_results
    else:
        print(f'Error: {response.status_code} - {response.text}')
        break

key,summary,status,created,sprint,reporter,email,assignee,team,rapyd_bu,department,story_points,issue_type = [],[],[],[],[],[],[],[],[],[],[],[],[]

for issue in issues:
    key.append(issue['key'])
    summary.append(issue['fields']['summary'])
    status.append(issue['fields']['status']['name'])
    created.append(issue['fields']['created'])
    sprint.append(issue['fields'].get('customfield_10113', None))
    reporter.append(issue['fields']['reporter']['displayName'] if issue['fields']['reporter'] else None)
    email.append(issue['fields']['reporter']['emailAddress'] if issue['fields']['reporter'] else None)
    assignee.append(issue['fields']['assignee']['displayName'] if issue['fields']['assignee'] else None)
    team.append(issue['fields']['customfield_10703']['value'] if issue['fields']['customfield_10703'] else None)
    rapyd_bu.append(issue['fields']['customfield_11671']['value'] if issue['fields']['customfield_11671'] else None)
    department.append(issue['fields']['customfield_10623']['value'] if issue['fields']['customfield_10623'] else None)
    story_points.append(issue['fields'].get('customfield_10595', None))
    issue_type.append(issue['fields']['issuetype']['name'])

data = {'issue_key':key,'summary':summary, 'status':status, 'created':created, 'sprint':sprint,
        'reporter':reporter, 'reporter_email':email, 'assignee':assignee, 'team':team, 'business_unit':rapyd_bu, 
        'department':department, 'story_points':story_points, 'issue_type': issue_type}

jira = pd.DataFrame(data)

jira['created'] = pd.to_datetime(jira['created'].str.split('T').str[0] + ' 00:00:00')

jira['sprint'] = jira['sprint'].apply(lambda x: x[0]['name'] if isinstance(x,list) and len(x) > 0 and 'name' in x[0] else None)

jira['story_points'] = pd.to_numeric(jira['story_points'])

# keep only relevant tickets
df = jira[(jira.assignee.isin(team_members)) | 
          (jira.team == 'Analytics') | 
          ((jira.team.isnull()) & (jira.assignee.isnull()))].copy()

df = df[df.status != "Won't Do"].reset_index(drop = True).copy()

df.head(3)

#### Conditions

In [ ]:
df.loc[(df.assignee.isin(team_members)) & (df.team == 'Analytics') & (df.story_points.isnull()), 'case'] = 'No story points'

df.loc[(df.assignee.isin(team_members)) & (df.team == 'Analytics') & (df.sprint.isnull()), 'case'] = 'No sprint'

df.loc[(df.assignee.isin(team_members)) & (df.team != 'Analytics'), 'case'] = 'Wrong team name'

df.loc[(df.assignee.isin(team_members)) & (df.team.isnull()), 'case'] = 'No team name'

df.loc[(df.assignee.isnull()) & (df.team == 'Analytics'), 'case'] = 'Backlog'

df.loc[(df.assignee.isnull()) & (df.team == 'Analytics') & df.sprint.notnull(), 'case'] = 'No assignee'

value_counts_2(df, 'case', drop_null = True)

In [ ]:
cols = ['case','issue_key','summary','status','created','sprint','reporter','assignee','team','story_points','issue_type']

final = df[df.case.notnull()][cols].sort_values(by = ['case','assignee'], ascending = False, ignore_index = True).copy()

final[final.status != 'Done'].sort_values(by = 'assignee')